# Panel irradiance interpolation

# Imports

In [1]:
import warnings
import glob
import os
import exiftool
import micasense.imageset as imageset
import micasense.capture as capture
import micasense.metadata as metadata
import numpy as np
print(np.version.version)

warnings.filterwarnings('ignore')

1.19.2


## 2.  Setting up the images directory 

In [2]:
os.listdir('Images')
images_dir = 'Images'


0.5714285714285714

## 3. Reading calibration panel images 

In [3]:
panel = sorted(glob.glob(images_dir + '/**/*.tif', recursive=True))

# First six images are the panel image captured before the fly
panelNames_start = panel[0:6]

# Last six images are the panel image captured after the fly
panelNames_end = panel[len(panel)-6:len(panel)]

# Reading capture class
panelcap_start = capture.Capture.from_filelist(panelNames_start)
panelcap_end = capture.Capture.from_filelist(panelNames_end)

# Reflectance by band, calibration values
panel_reflectance_by_band = panelcap_start.panel_albedo()
panel_reflectance_by_band

# Irradiance by band obtained by the calibration panel before the fly
panel_irradiance_start = np.array(
    panelcap_start.panel_irradiance(panel_reflectance_by_band))

# Irradiance by band obtained in the DLS sensor
dls_irradiance_start = np.array(panelcap_start.dls_irradiance()[:-1])

# Correction fraction
dls_correction_start = panel_irradiance_start/dls_irradiance_start
dls_correction_start

# Reading metadata and getting the panel capture time
meta_start = metadata.Metadata(panelNames_start[0])
Time_start = meta_start.utc_time().hour + (meta_start.utc_time().minute/60) + \
    (meta_start.utc_time().second/3600)


panelcap_end = capture.Capture.from_filelist(panelNames_end)
panel_irradiance_end = np.array(
    panelcap_end.panel_irradiance(panel_reflectance_by_band))
dls_irradiance_end = np.array(panelcap_end.dls_irradiance()[:-1])
dls_correction_end = panel_irradiance_end/dls_irradiance_end
dls_correction_end
panelcap_end.plot_undistorted_reflectance(
    dls_irradiance_end*dls_correction_end)
panelcap_end.plot_panels()
meta_end = metadata.Metadata(panelNames_end[0])
Time_end = meta_end.utc_time().hour + (meta_end.utc_time().minute / 60) + \
    (meta_end.utc_time().second/3600)
meta_end.get_all()

# Linear interpolation
m = (panel_irradiance_end-panel_irradiance_start)/(Time_end-Time_start)


def lin_interp(time):
    return (m*(time-Time_start)+panel_irradiance_start)*100


panel_irradiance_end, Time_end
panel_irradiance_start, Time_start

meta_start.get_all()
panelcap_start.dls_irradiance()
panelcap_end.dls_irradiance()

# plt.plot([Time_start,Time_end],[panel_irradiance_start[0],panel_irradiance_end[0]],'bo-')
# #plt.gcf().autofmt_xdate()
# plt.xlabel('Time (decimal hour)')
# plt.ylabel('Irradiance ($W/m^2/nm$)')
# plt.show();

os.listdir()
#images_dir = os.path.join(images_dir)
imgset = imageset.ImageSet.from_directory(images_dir)

for cap in imgset.captures:
    print ("Opened Capture {} with bands {}".format(
        cap.uuid, [str(band) for band in cap.band_names()]))

imgset_ls = imgset.as_nested_lists()[0]
imgset_ls

dat = []
irr = []
for i in range(0, len(imgset_ls)):
    dat.append(imgset_ls[i][0].hour + (imgset_ls[i]
                                       [0].minute/60)+(imgset_ls[i][0].second/3600))
    irr.append(imgset_ls[i][8:13])

irr_interp = []
for i in range(0, len(imgset_ls)):
    irr_interp.append(lin_interp(dat[i]))
irr_interp
irr_interp_ls = np.asarray(irr_interp).flatten()
irr_interp_ls_b = []
for i in range(0, len(irr_interp_ls)):
    irr_interp_ls_b.append(bytes(str(irr_interp_ls[i]), 'utf8'))


images = sorted(glob.glob(images_dir + '/**/*[12345].tif', recursive=True))
images_b = []
for i in range(0, len(images)):
    images_b.append(bytes(images[i], 'utf8'))

len(images)

irr_interp_ls_b[0]
et = exiftool.ExifTool()
et.start()
for i in range(0, len(images_b)):
    et.execute(b'-Irradiance=' +
               irr_interp_ls_b[i], images_b[i], b'-overwrite_original')
    et.execute(b'-HorizontalIrradiance=' +
               irr_interp_ls_b[i], images_b[i], b'-overwrite_original')







In [4]:
panel_reflectance_by_band

[0.5365, 0.5348, 0.53264, 0.5292, 0.53166]